In [1]:
using Pkg
Pkg.activate("..")
Pkg.instantiate()

  Activating project at `~/AwID/MLP`


In [2]:
using JLD2
X_train = load("../data/imdb_dataset_prepared.jld2", "X_train")
y_train = load("../data/imdb_dataset_prepared.jld2", "y_train")
X_test = load("../data/imdb_dataset_prepared.jld2", "X_test")
y_test = load("../data/imdb_dataset_prepared.jld2", "y_test")
nothing

In [ ]:
using MLP.NeuralNetwork, MLP.AutoDiff, Printf

model = Chain(
    Dense(size(X_train, 1), 32, relu),
    Dense(32, 1, sigmoid)
)
opt = Adam()

batch_size = 64

placeholder_x_data = similar(X_train, size(X_train, 1), batch_size)
placeholder_y_data = similar(y_train, 1, batch_size)

x_input_node = Variable(placeholder_x_data)
y_true_node = Constant(placeholder_y_data)
loss_node_static, y_pred_node_static = binary_crossentropy(model, x_input_node, y_true_node, batch_size=batch_size)
computation_order_static = topological_sort(loss_node_static)

x_test_node_static = Variable(X_test)
y_test_node_static = Constant(y_test)
test_loss_node_static, test_y_pred_node_static = binary_crossentropy(model, x_test_node_static, y_test_node_static, batch_size=size(y_test, 2))
test_order_static = topological_sort(test_loss_node_static)

epochs = 5
for epoch in 1:epochs
    epoch_total_loss = 0.0f0
    epoch_total_acc = 0.0f0
    num_processed_batches = 0

    epoch_batches = get_epoch_batches(X_train, y_train, batch_size=batch_size, do_shuffle=true)

    t_elapsed = @elapsed begin
        for (x_batch, y_batch) in epoch_batches
            x_input_node.output = x_batch
            y_true_node.output = y_batch

            current_loss_value = forward!(computation_order_static)
            batch_acc_value = accuracy(y_pred_node_static.output, y_batch)

            epoch_total_loss += current_loss_value
            epoch_total_acc += batch_acc_value
            num_processed_batches += 1

            backward!(computation_order_static)
            update!(opt, parameters(model))
        end

        train_loss = epoch_total_loss / num_processed_batches
        train_acc = epoch_total_acc / num_processed_batches

        test_loss_value = forward!(test_order_static)
        test_acc_value = accuracy(test_y_pred_node_static.output, y_test)
    end

    println(@sprintf("Epoch: %d (%.2fs) \tTrain: (l: %.2f, a: %.2f) \tTest: (l: %.2f, a: %.2f)",
        epoch, t_elapsed, train_loss, train_acc, test_loss_value, test_acc_value))
end

Epoch: 1 (21.23s) 	Train: (l: 0.65, a: 0.80) 	Test: (l: 0.58, a: 0.85)
Epoch: 2 (7.34s) 	Train: (l: 0.46, a: 0.92) 	Test: (l: 0.44, a: 0.87)
Epoch: 3 (8.28s) 	Train: (l: 0.30, a: 0.94) 	Test: (l: 0.36, a: 0.87)
Epoch: 4 (8.54s) 	Train: (l: 0.20, a: 0.96) 	Test: (l: 0.33, a: 0.87)
Epoch: 5 (6.87s) 	Train: (l: 0.15, a: 0.98) 	Test: (l: 0.32, a: 0.87)
